# Import SQLite and Datetime Objects

In [ ]:
#Import sqlite
import sqlite3

In [ ]:
#import datetime objects
import datetime
from datetime import time

In [ ]:
#import DB
db=sqlite3.connect("herewegoDB.db")

In [ ]:
#create cursor
cursor = db.cursor()

In [ ]:
#to test the connection
cursor.execute('''select * FRom users''')
p= cursor.fetchall()
for j in p:
    print(p)


In [ ]:
#Helper function
#determines the role of a user provided user is in the database
def get_user_role(username):
    role = ""
    cursor.execute('''
        SELECT user_role FROM users
        WHERE username = ?
        ''',
        (username,)
        )
    role = cursor.fetchone()
    if role is None:
        return "User '%s' is not found or user does not have a role" %username
    else:
        return role[0]

In [ ]:
get_user_role("aditya")

In [ ]:
#Helper function
def change_event_capacity(event_name,capacity):
    cursor.execute('''
            UPDATE events SET event_capacity = ?
            WHERE event_name = ?
        ''', (capacity,event_name))
    db.commit()
    cursor.execute('''
                    SELECT event_name, event_capacity 
                    FROM events WHERE event_name=?''', (event_name,))
    event_detail = cursor.fetchone()
    if event_detail is None:
        return "Event name '%s' has not been created. Please ensure the right event has been entered or created and try again" %event_name
    else:
        return event_detail

In [ ]:
change_event_capacity("singing",1)

# Add a User. For admin only

In [ ]:
def add_user(user_account,username,first_name,last_name,age,email,user_role,user_phone):
    try:
        creation_date = datetime.datetime.now().strftime("%Y-%m-%d")
        if get_user_role(user_account) != 'admin':
            return "User %s doesn't exist or does not have appropriate permission to add other users" %user_account
        else:
            cursor.execute('''
                            INSERT INTO users
                            VALUES(?,?,?,?,?,?,?,?)
                ''',(username,first_name,last_name,age,email,user_role,user_phone,creation_date)
                )
            db.commit()
            cursor.execute('''
                            SELECT * FROM users
                ''')
            users = cursor.fetchall()
            for user in users:
                print(user)
    except sqlite3.Error as e:
        print("Error {" + e.args[0] + "}") 

In [ ]:
add_user("darasimi","bobo","jon","snow",25,"jon.snow@got.com","user","21597648")

In [ ]:
#Helper Function
def fetch_venue(venue_name):
    cursor.execute("SELECT * FROM venues WHERE venue_name=?",(venue_name,))
    all= cursor.fetchall()
    return(all[0])
        

In [ ]:
fetch_venue('rainey')

# Add a venue. For admin only

In [ ]:
#add a venue, 
#only admin can add or update venues#added by aditya

def add_venue(user_account,venue_name,venue_open,venue_close,zip_code,city,address):
    user_role = get_user_role(user_account)
    try:
        if user_role!='admin':
            return "You do not have rights to create or update venues"
        else:            
            cursor.execute('''
            insert into venues(venue_name,venue_open,venue_close,zip_code,city,address)
            values(?,?,?,?,?,?)
            ''',(venue_name,venue_open,venue_close,zip_code,city,address))
            db.commit()
            cursor.execute("SELECT * FROM venues WHERE venue_name=?",(venue_name,))
            all_venues = cursor.fetchall()
            for p in all_venues:
                print(p)
            
    except sqlite3.Error as e:
         print("Error {" + e.args[0] + "}")
        

In [ ]:
add_venue('aditya',"Rutwicks House",'8:00','20:00',"78751","Austin","31156 Helms Street")

# List events at a venue given the date/time

In [ ]:
#List events at a venue by date/time
def display_venue_events(venue_id,event_date,event_start):
    try:
        cursor.execute('''select v.venue_name,e.event_name,e.event_description,e.event_capacity,e.event_start,e.event_end from events e, venues v where e.venue_id = v.venue_id 
        and e.venue_id = ? and e.event_date = ? and e.event_start = ?
        ''',(venue_id,event_date,event_start,))
        venues = cursor.fetchall()
        if not venues or venues is None:
            print("No events scheduled at the venue")
        else:
            print(venues)
    except sqlite3.Error as e:
        print("Error {" + e.args[0] + "}")
            



In [ ]:
display_venue_events(2,"2019-09-21","09:00")

In [ ]:
#Helper function
#check current event capacity # added by aditya
#it accepts event_id as the input and returns the current event capacity
def check_capacity(event_id):
    try:
        cursor.execute('''select event_capacity FRom events
                       where event_id = ?
                       ''',(event_id,))
        current_cap = cursor.fetchone()
        return (current_cap[0])
    except sqlite3.Error as e:
        print("Error {" + e.args[0] + "}")

In [ ]:
check_capacity(2)

# A user joins an event

In [ ]:
# Function implemented to join an event.
# added by aditya. It accepts the user id and event id as input
def join_event(username,event_id):
    msg = "sucess"    
    try:
        event_cap = check_capacity(event_id)
        if event_cap >0:
            #print Make entry into user events table and reduce the flag count
            cursor.execute('''select username from users where username=?
                        ''',(username,))
            user_result = cursor.fetchone()
            username = user_result[0]
            #make entries into the user events table
            cursor.execute('''insert into user_events(username,event_id)
                              values(?,?)''',(username,event_id,))
            #increase the event capacity count            
            cursor.execute('''update events set event_capacity=event_capacity-1 
                           where event_id = ?
                           ''',(event_id,))
            db.commit()
            return msg
            
        else:
            print ("Please join later")
    except sqlite3.Error as e:
        print("Error {" + e.args[0] + "}")
        

In [ ]:
join_event("jane",2)

# Admin removes an event

In [ ]:
#remove an event. Added by Aditya # this feature only lets the admin remove an event.
# it accepts the event id and username as input.
# Steps:
# 1. Delete the entries from user_events table
# 2. Delete the entries from the events table


def remove_event(event_id,username):
    msg = "Event Succesfully Deleted"
    try:
              
        if get_user_role(username)=="admin":
            cursor.execute('''delete from user_events where event_id =?
                           ''',(event_id,))
            cursor.execute('''delete from events where event_id =?
                           ''',(event_id,))
            db.commit()
            return msg
        else:
            print("You are not authorized to perform this action")
    
    except sqlite3.Error as e:
        print("Error {" + e.args[0] + "}")





In [ ]:
remove_event(5,"darasimi")

In [ ]:
#Helper function
#User opts out of an event. Added by Aditya # this feature allows the users to exit an event.
#1. This should remove the user entry from the user_events table for a particular event
#2. Event capacity should be increased by one everytime a user quits an event.
def quit_event(username,event_id):
    msg = "User removed from the event succesfully"
    
    try:
        cursor.execute('''select username From users where username = ?
                      ''',(username,))
        user_result = cursor.fetchone()
        username = user_result[0]
        
        
        cursor.execute('''delete from user_events where username =?
                       ''',(username,))
        
        cursor.execute('''update events set event_capacity=event_capacity+1
                       where event_id =?
                       ''',(event_id,))
        db.commit()
        return msg
        
    except sqlite3.Error as e:
        print("Error {"+ e.args[0] +"}")

In [ ]:
quit_event("jane",2)

In [ ]:
#Helper function
#Check if a venue is available at a given date and time slot. Any user or admin can call this function.
#If the slot is available it returns boolean value True else returns False.
#Version 1.0 created by Aditya

def venue_availables(venue_id,venue_date,start_time): 
    try: 
        cursor.execute('''SELECT v.venue_name,vc.slot_start,vc.slot_end,vc.is_available,v.zip_code,v.city,v.address
        FROM venue_calendar vc,venues v WHERE vc.is_available = 'Y' and vc.venue_id = v.venue_id and
        vc.venue_id=?  and vc.venue_date = ? and vc.slot_start =?
        ''',(venue_id,venue_date,start_time,))
        result = cursor.fetchone()
        #print(result[0])
        if  result is None:
            return False
        else:
            return True
            
    except sqlite3.Error as e:
        print("Error {"+ e.args[0] +"}")



In [ ]:
venue_available(2,"2019-07-29","1:00")

In [ ]:
#Helper function
#Function that adds an hour to an exisiting hour specificed
def add_hour(start_time):
    hour = start_time.split(':')[0]
    minute = start_time.split(':')[1]
    hour_int = int(hour)
    minute_int = int(minute)
    hour_int_end = (hour_int+1) if hour_int < 23 else 0 
    end_time = time(hour_int_end,minute_int)
    end_time_final = end_time.strftime("%H:%M")
    return end_time_final

In [ ]:
print(add_hour("23:00"))

In [ ]:
#Helper function
#Checks if a particular venue is available with a given venue id, start time and Date of event
def venue_available(venue_id,start_time,event_date):
    try:
        cursor.execute('''
                        SELECT venue_id
                        FROM venue_calendar
                        WHERE venue_id = ? AND slot_start = ? AND venue_date = ?
        ''',(venue_id,start_time,event_date))
        
        result = cursor.fetchone()
        if result is None:
            return True
        else:
            return False
    except sqlite3.Error as e:
            print("Error {" + e.args[0] + "}")
    

In [ ]:
venue_available(2,"09:00","2019-09-21")

# A user starts/creates an event

In [ ]:
#Functionality to create or start an event. Once event is created its added to the events_table and calendar.
def start_event(name,city,event_type,start_time,capacity,venue_id,username,description,event_date):
    try:
        end_time = add_hour(start_time)
        creation_date = datetime.datetime.now().strftime("%Y-%m-%d")
        not_available = "N"
        if venue_available(venue_id,start_time,event_date):
            cursor.execute('''
                INSERT INTO events(event_name,event_city,event_type,event_start,event_end,event_capacity,venue_id,creation_date,username,event_description,event_date)
                VALUES(?,?,?,?,?,?,?,?,?,?,?)                         
                ''',(name,city,event_type,start_time,end_time,capacity,venue_id,creation_date,username,description,event_date))
            cursor.execute('''
                            INSERT INTO venue_calendar(venue_id,venue_date,slot_start,slot_end,is_available)
                            VALUES(?,?,?,?,?)                     
            ''',(venue_id,event_date,start_time,end_time,not_available))
            db.commit()
            cursor.execute('''SELECT * FROM events WHERE event_name = ?''', (name,))
            events = cursor.fetchall()
            for event in events:
                print(event)
        else:
            return "Event '%s' for %s on %s is not available. Please pick a different time" %(name,start_time,event_date)
    except sqlite3.Error as e:
        print("Error {"+ e.args[0] +"}")  

In [ ]:
start_event("soccer for life","Austin","soccer","09:00",25,2,"stevendar","if you like to have fun","2019-09-22")

# Displays all the venues where a particular time slot is available on a given date

In [ ]:
#Displays all the venues where a particular time slot is available.
#Paramneters are the start time you would like the event to be and the Date of the event you are looking to book for
def timeslot_availability(start_time,event_date):
    try:
        cursor.execute('''
                SELECT v.venue_name
                FROM venues v
                WHERE NOT EXISTS (SELECT e.venue_id FROM events e WHERE v.venue_id = e.venue_id 
                AND e.event_start = ? AND e.event_date = ? ) AND v.venue_open <= strftime('%H:%M',?)
                AND v.venue_close >= strftime('%H:%M',?) 
                
        ''',(start_time,event_date,start_time,start_time))
        results = cursor.fetchall()
        if results:
            for result in results:
                print(result)
        else:
            print("There's no venue available for this time slot")
    except sqlite3.Error as e:
        print("Error {"+ e.args[0] +"}")

In [ ]:
timeslot_availability("09:00","2019-09-21")

In [ ]:
#Helper function
#Time gap function that returns the range of times between a start and end date. Source of reference("https://stackoverflow.com/questions/40815648/get-all-the-times-in-between-two-datetime-in-list")
def time_gap(event_start,event_end):
    reserved_time = []
    available_time = []
    fmt = '%H:%M'
    strp_a = datetime.datetime.strptime(event_start,fmt)
    strp_b = datetime.datetime.strptime(event_end,fmt) 
    while strp_a <= strp_b:
        available_time.append(str(strp_a))
        strp_a += datetime.timedelta(hours=1)
    for time in available_time:
        val = datetime.datetime.strptime(time, '%Y-%m-%d %H:%M:%S')
        time = val.strftime("%H:%M")
        reserved_time.append(time)
    return reserved_time
        
    

# Displays timeslot availability at a particular venue on a given date


In [ ]:
#Functionality to display timeslot at a particular venue. Parameters are Venue ID and Date of Event
def venue_timeslot(venue_id,event_date):
    try:
        all_dates =[]
        cursor.execute('''
                SELECT venue_open FROM venues
                WHERE venue_id = ?
        ''',(venue_id,))
        open_time = cursor.fetchone()
        
        cursor.execute('''
                SELECT venue_close FROM venues
                WHERE venue_id = ?
        ''',(venue_id,))
        
        end_time = cursor.fetchone()
        
        start_time = open_time[0]
        close_time = end_time[0]

        cursor.execute('''
                SELECT event_start FROM events e, venues v
                WHERE e.venue_id = ? AND e.event_date = ? AND e.venue_id = v.venue_id
        ''',(venue_id,event_date))

        reserved_dates = cursor.fetchall()
        
        for dates in reserved_dates:
            all_dates.append(dates[0])
            
        venue_hours = time_gap(start_time,close_time)
        if reserved_dates:
            main_list = [item for item in venue_hours if item not in all_dates ]
            return main_list
        else:
            return venue_hours
    except TypeError as t:
        print("Venue does exist, please check venue and try again")
    
    except sqlite3.Error as e:
        print("Error {"+ e.args[0] +"}")
    
    
    
    

In [ ]:
venue_timeslot(2,"2019-09-22")

In [ ]:
db.close()